In [0]:

# Load the section metadatas
import json
import pandas as pd

with open("/Volumes/main/casml/raw_data/book-metadata.json", "r") as f:
    book_metadata = json.load(f)

# Map page-nums to sections
def map_page_to_section(metadata):
    page_map = {}
    
    for section, sec_data in metadata.items():
        sec_start = sec_data.get("page_start")
        sec_end = sec_data.get("page_end")
        subsections = sec_data.get("subsections", {})

        for page in range(sec_start, sec_end + 1):
            page_map[page] = {"section": section}

        for subsec, subsec_data in subsections.items():
            sub_start = subsec_data.get("page_start")
            sub_end = subsec_data.get("page_end")
            for page in range(sub_start, sub_end + 1):
                page_map[page] = {"section": f'{section}/{subsec}'}

    return page_map

page_to_section = map_page_to_section(book_metadata)

# Convert page_to_section dict to a DataFrame
page_section_df = pd.DataFrame([
    {"page_num": page, "section": info["section"]}
    for page, info in page_to_section.items()
])

# Create Spark DataFrame
page_section_sdf = spark.createDataFrame(page_section_df)

# Create a temporary SQL table
page_section_sdf.createOrReplaceTempView("pag_to_section_temp")



In [0]:
%sql
USE main.casml;
-- DROP TABLE knowledge_base;
CREATE TABLE IF NOT EXISTS knowledge_base (
  id BIGINT GENERATED ALWAYS AS IDENTITY,
  page_num INT,
  content STRING,
  section STRING
)
TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [0]:
%sql
INSERT OVERWRITE TABLE knowledge_base (page_num, content)
SELECT
first_value(e.value:bbox[0]:page_id) AS page_num,
string_agg(e.value:content) AS content
FROM (
  SELECT ai_parse_document(content) AS parsed_document, path
  FROM READ_FILES('/Volumes/main/casml/raw_data/book-1-20.pdf', format => 'binaryFile')
) AS f,
LATERAL variant_explode(f.parsed_document:document:elements) AS e
GROUP BY e.value:bbox[0]:page_id::int, f.path
HAVING page_num::int > 7




num_affected_rows,num_inserted_rows
12,12


In [0]:
%sql
MERGE INTO knowledge_base AS kb
USING pag_to_section_temp AS pts
ON kb.page_num = pts.page_num
WHEN MATCHED THEN
  UPDATE SET kb.section = pts.section;

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
12,12,0,0


In [0]:
%sql
SELECT * from knowledge_base

id,page_num,content,section
48,13,"2 Prefacestudies and current and emerging research. The text also includes coverage of the DSM-5 in examinations of psychological disorders. Psychology 2e incorporates discussions that reflect the diversity within the discipline, as well as the diversity of cultures and communities across the globe.Coverage and scopeThe first edition of Psychology has been used by thousands of faculty and hundreds of thousands of students since its publication in 2015. OpenStax mined our adopters' extensive and helpful feedback to identify the most significant revision needs while maintaining the organization that many instructors had incorporated into their courses. Specific surveys, pre-revision reviews, and customization analysis, as well as analytical data from OpenStax partners and online learning environments, all aided in planning the revision.The result is a book that thoroughly treats psychology's foundational concepts while adding current and meaningful coverage in specific areas. Psychology 2e retains its manageable scope and contains ample features to draw learners into the discipline.Structurally, the textbook remains similar to the first edition, with no chapter reorganization and very targeted changes at the section level.Chapter 1: Introduction to PsychologyChapter 2: Psychological ResearchChapter 3: BiopsychologyChapter 4: States of ConsciousnessChapter 5: Sensation and PerceptionChapter 6: LearningChapter 7: Thinking and IntelligenceChapter 8: MemoryChapter 9: Lifespan DevelopmentChapter 10: Motivation and EmotionChapter 11: PersonalityChapter 12: Social PsychologyChapter 13: Industrial-Organizational PsychologyChapter 14: Stress, Lifestyle, and HealthChapter 15: Psychological DisordersChapter 16: Therapy and TreatmentChanges to the Second EditionOpenStax only undertakes second editions when significant modifications to the text are necessary. In the case of Psychology 2e, user feedback indicated that we needed to focus on a few key areas, which we have done in the following ways.Content revisions for clarity, accuracy, and currencyThe revision plan varied by chapter based on need. Some chapters were significantly updated for conceptual coverage, research-informed data, and clearer language. In other chapters, the revisions focused mostly on currency of examples and updates to statistics.Over 210 new research references have been added or updated in order to improve the scholarly underpinnings of the material and broaden the perspective for students. Dozens of examples and feature boxes have been changed or added to better explain concepts and/or increase relevance for students.Research replication and validityTo engage students in stronger critical analysis and inform them about research reproducibility, substantial coverage has been added to the research chapter and strategically throughout the textbook whenever keyAccess for free at openstax.org",Introduction to Psychology/History of Psychology
47,11,Access for free at openstax.org,Introduction to Psychology/History of Psychology
46,15,"4 Prefaceand Social Norms,"" and ""Conditioning and OCD.""Art, interactives, and assessments that engageOur art program is designed to enhance students' understanding of psychological concepts through simple, effective graphs, diagrams, and photographs. Psychology 2e also incorporates links to relevant interactive exercises and animations that help bring topics to life. Selected assessment items touch directly on students' lives.Link to Learning features direct students to online interactive exercises and animations that add a fuller context to core content and provide an opportunity for application.Personal Application Questions engage students in topics at a personal level to encourage reflection and promote discussion.Additional ResourcesStudent and Instructor ResourcesWe've compiled additional resources for both students and instructors, including Getting Started Guides, an instructor solution guide, a

In [0]:
%pip install -U -qqqq mlflow>=3.1.1 langchain=0.4.0.dev0 langgraph databricks-langchain pydantic databricks-agents unitycatalog-langchain[databricks] uv databricks-feature-engineering==0.12.1
dbutils.library.restartPython()



Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import time

def endpoint_exists(vsc, vs_endpoint_name):
  try:
    return vs_endpoint_name in [e['name'] for e in vsc.list_endpoints().get('endpoints', [])]
  except Exception as e:
    #Temp fix for potential REQUEST_LIMIT_EXCEEDED issue
    if "REQUEST_LIMIT_EXCEEDED" in str(e):
      print("WARN: couldn't get endpoint status due to REQUEST_LIMIT_EXCEEDED error. The demo will consider it exists")
      return True
    else:
      raise e

def wait_for_vs_endpoint_to_be_ready(vsc, vs_endpoint_name):
  for i in range(180):
    try:
      endpoint = vsc.get_endpoint(vs_endpoint_name)
    except Exception as e:
      #Temp fix for potential REQUEST_LIMIT_EXCEEDED issue
      if "REQUEST_LIMIT_EXCEEDED" in str(e):
        print("WARN: couldn't get endpoint status due to REQUEST_LIMIT_EXCEEDED error. Please manually check your endpoint status")
        return
      else:
        raise e
    status = endpoint.get("endpoint_status", endpoint.get("status"))["state"].upper()
    if "ONLINE" in status:
      return endpoint
    elif "PROVISIONING" in status or i <6:
      if i % 20 == 0: 
        print(f"Waiting for endpoint to be ready, this can take a few min... {endpoint}")
      time.sleep(10)
    else:
      raise Exception(f'''Error with the endpoint {vs_endpoint_name}. - this shouldn't happen: {endpoint}.\n Please delete it and re-run the previous cell: vsc.delete_endpoint("{vs_endpoint_name}")''')
  raise Exception(f"Timeout, your endpoint isn't ready yet: {vsc.get_endpoint(vs_endpoint_name)}")

def index_exists(vsc, endpoint_name, index_full_name):
    try:
        vsc.get_index(endpoint_name, index_full_name).describe()
        return True
    except Exception as e:
        if 'RESOURCE_DOES_NOT_EXIST' not in str(e):
            print(f'Unexpected error describing the index. This could be a permission issue.')
            raise e
    return False
  
def wait_for_index_to_be_ready(vsc, vs_endpoint_name, index_name):
  for i in range(180):
    idx = vsc.get_index(vs_endpoint_name, index_name).describe()
    index_status = idx.get('status', idx.get('index_status', {}))
    status = index_status.get('detailed_state', index_status.get('status', 'UNKNOWN')).upper()
    url = index_status.get('index_url', index_status.get('url', 'UNKNOWN'))
    if "ONLINE" in status:
      return
    if "UNKNOWN" in status:
      print(f"Can't get the status - will assume index is ready {idx} - url: {url}")
      return
    elif "PROVISIONING" in status:
      if i % 40 == 0: print(f"Waiting for index to be ready, this can take a few min... {index_status} - pipeline url:{url}")
      time.sleep(10)
    else:
        raise Exception(f'''Error with the index - this shouldn't happen. DLT pipeline might have been killed.\n Please delete it and re-run the previous cell: vsc.delete_index("{index_name}, {vs_endpoint_name}") \nIndex details: {idx}''')
  raise Exception(f"Timeout, your index isn't ready yet: {vsc.get_index(index_name, vs_endpoint_name)}")

In [0]:
VECTOR_SEARCH_ENDPOINT_NAME = "casml_vs_endpoint"

from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient(disable_notice=True)

if not endpoint_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME):
    endpoints = vsc.list_endpoints()
    if len(endpoints):
        endpoint_names = [ep['name'] for ep in endpoints['endpoints']]
        for name in endpoint_names:
            vsc.delete_endpoint(name)

    vsc.create_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME, endpoint_type="STANDARD")

wait_for_vs_endpoint_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME)
print(f"Endpoint named {VECTOR_SEARCH_ENDPOINT_NAME} is ready.")

Endpoint named casml_vs_endpoint is ready.


In [0]:
from databricks.sdk import WorkspaceClient

#The table we'd like to index
catalog = "main"
dbName = "casml"
source_table_fullname = f"{catalog}.{dbName}.knowledge_base"
# Where we want to store our index
vs_index_fullname = f"{catalog}.{dbName}.knowledge_base_vs_index"

if not index_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname):
  print(f"Creating index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
  vsc.create_delta_sync_index(
    endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
    index_name=vs_index_fullname,
    source_table_name=source_table_fullname,
    pipeline_type="TRIGGERED",
    primary_key="id",
    embedding_source_column='content', #The column containing our text
    embedding_model_endpoint_name='databricks-gte-large-en', #The embedding endpoint used to create the embeddings
    columns_to_sync=['page_num', 'section']
  )
  #Let's wait for the index to be ready and all our embeddings to be created and indexed
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
else:
  #Trigger a sync to update our vs content with the new data saved in the tableoug
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
  vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname).sync()

print(f"index {vs_index_fullname} on table {source_table_fullname} is ready")

index main.casml.knowledge_base_vs_index on table main.casml.knowledge_base is ready


In [0]:
question = "What was the main contribution the 'Chain-of-Thought Prompting Elicits Reasoning in Large Language Models' paper"

results = vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname).similarity_search(
  query_text=question,
  columns=["id", "content", 'page_num', 'section'],
  num_results=1)
  
docs = results.get('result', {}).get('data_array', [])

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


In [0]:
from agent import AGENT 

import ast
import re
import json
import io
import csv

def parse_doc_string(doc_string: str) -> dict | None:
    # Find all occurrences of the document's contents.
    pattern = re.compile(r"metadata=({.*?}), page_content='(.*?)'")
    matches = pattern.findall(doc_string)

    # Iterate through the matches and parse each one
    parsed_list = []
    for metadata_str, page_content_str in matches:
        try:
            # Safely parse the metadata string into a dictionary
            metadata = ast.literal_eval(metadata_str)
            
            # Decode the page content to correctly handle escape sequences like \n
            page_content = page_content_str.encode().decode('unicode_escape')
            
            # Append the structured dictionary to our list
            parsed_list.append({
                'metadata': metadata,
                'page_content': page_content
            })
        except (ValueError, SyntaxError) as e:
            print(f"Skipping a malformed document part due to: {e}")

    # Print the final list of dictionaries with nice formatting
    return parsed_list

def parse_agent_output(answer):
    if len(answer.output) < 3:
        return None
    for i in range(len(answer.output)):
        print(answer.output[i])

    results = {}
    parsed_list = parse_doc_string(answer.output[1].output if len(answer.output) > 1 else '')
    context = ""

    for doc_page_dict in parsed_list:
        context += doc_page_dict["page_content"]
        context += "\n\n"

    final_answer_idx = -1
    for i in range(2, len(answer.output)):
        if answer.output[i].type == "message":
            final_answer_idx = i

    if final_answer_idx != -1:
        content = answer.output[final_answer_idx].content[0]
        if "text" in content.keys():
            results['answer'] = content["text"]
        else:
            results['answer'] = ""
    else:
        results['answer'] = ""

    results['references'] = {"sections": [doc_page_dict["metadata"]["section"] for doc_page_dict in parsed_list],
                                "pages": [int(doc_page_dict["metadata"]["page_num"]) for doc_page_dict in parsed_list]}
    
    return results

def parsed_results_to_csv(data):
    fieldnames = data[0].keys()

    with open("/Volumes/main/casml/raw_data/output.csv", "w", newline='', encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

# Read all queries
with open("/Volumes/main/casml/raw_data/queries.json", "r") as f:
    queries = json.load(f)

results = []
for i, query in enumerate(queries):
    print(f'Query {i}')
    query_id = query['query_id']
    result = AGENT.predict({"input":[{"role": "user", "content": query['question']}]})
    parsed_result = parse_agent_output(result)

    if parsed_result:
        parsed_result['query_id'] = query_id
        results.append(parsed_result)

if len(results) > 0:
    parsed_results_to_csv(results)
else:
    raise Exception("No results obtained")


2025/10/25 19:42:09 WARNING mlflow.utils.autologging_utils: MLflow langchain autologging is known to be compatible with 0.3.3 <= langchain, but the installed version is 0.4.0.dev0. If you encounter errors during autologging, try upgrading / downgrading langchain to a compatible version, or try upgrading MLflow.
/databricks/python/lib/python3.12/site-packages/databricks/connect/session.py:477: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)
2025/10/25 19:42:10 WARNING mlflow.pyfunc: You have manually traced predict with @mlflow.trace, but this is unnecessary with ResponsesAgent subclasses. You can remove the @mlflow.trace decorator and it will be automatically traced.
2025/10/25 19:42:10 WARNING mlflow.pyfunc: You have manually traced predict_stream with @mlflow.trace, but this i

AGENT llm_endpoint_name:  databricks-meta-llama-3-1-8b-instruct
Model Config:  <mlflow.models.model_config.ModelConfig object at 0xffdf4fd5d460>
Query 0
Query 1
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.
type='function_call' id='run--ef98fd4b-702a-4b32-88e7-654dda4653e5' call_id='call_8b9305cb-a2f7-4087-bc84-b93f265291aa' name='main__casml__knowledge_base_vs_index' arguments='{"query": "basic parts of a neuron", "filters": null}'
type='function_call_output' call_id='call_8b9305cb-a2f7-4087-bc84-b93f265291aa' output='[Document(metadata={\'section\': \'Introduction to Psychology/What Is Psychology?\', \'page_num\': 8.0, \'id\': 27.0}, page_content=\'Critical Thinking Questions 246\\nPersonal Application Questions 246CHAPTER 8Memory 247\\nIntroduction 247\\n8.1 How Memory Functions 248\\n8.2 Parts of the Brain Involved with 

[Trace(trace_id=tr-1a8a06f8ec326dc147106bf4e0882cb4), Trace(trace_id=tr-a5021129d1536cbf5c798c266c979efa), Trace(trace_id=tr-a8d0ace6fc3c08f5fd295c6d092a5230), Trace(trace_id=tr-6230a8079717b916ba92cf425e68709d), Trace(trace_id=tr-f738deaa12e8b8ee031c02e876885e93), Trace(trace_id=tr-f52bcdf21382e2f3bd2b90be56a30369), Trace(trace_id=tr-4306cb4cfae3020c62dc3ab791b80556), Trace(trace_id=tr-cabb539cdf1a5730be3aaecf192699e0), Trace(trace_id=tr-cf70499479d518c886f4bba9cef0aebd), Trace(trace_id=tr-a479798ec285e133dd408d804d9d502c)]

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-8945825502578450>, line 86
     84 query_id = query['query_id']
     85 result = AGENT.predict({"input":[{"role": "user", "content": query['question']}]})
---> 86 parsed_result = parse_agent_output(result)
     88 if parsed_result:
     89     parsed_result['query_id'] = query_id

File <command-8945825502578450>, line 42, in parse_agent_output(answer)
     39     print(answer.output[i])
     41 results = {}
---> 42 parsed_list = parse_doc_string(answer.output[1].output if len(answer.output) > 1 else '')
     43 context = ""
     45 for doc_page_dict in parsed_list:

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-e8bef8e5-3e22-4286-bec0-fc0a0b52352d/lib/python3.12/site-packages/pydantic/main.py:1026, in BaseModel.__getattr__(self, item)
   1023     return super().__getattribute__(item)  # Raises AttributeError if appropria

In [0]:
%pip show langchain
%pip show databricks-langchain